In [ ]:
import os
from dotenv import load_dotenv
import whisper
from pyannote.audio import Pipeline
from pydub import AudioSegment
from pyannote.core import Segment
from pyannote.audio import Audio
## HugingFaceのAPIトークンを設定
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

In [ ]:
# モデルの読み込み(確認用)
model = whisper.load_model('small') 
result = model.transcribe('./meigen01.m4a')
print(result['text'])

In [ ]:
# 音声のモデリング準備
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=hf_token)

In [ ]:
# 音声ファイルの変換(必要)
audio = AudioSegment.from_file("./meigen01.m4a", format="m4a")
# wavファイルとして保存
audio.export("meigen01.wav", format="wav")

In [ ]:
# file_pathを指定 (wav形式)
audio_file="meigen01.wav"
# ダイアライゼーション実行
diarization = pipeline(audio_file,num_speakers=2)
# 音声の読み込み (PyAnnote用)
audio = Audio(sample_rate=16000, mono=True)
# 音声ファイルの正確な長さを取得（フレーム数 → 秒）
waveform_all, sample_rate = audio(audio_file)
duration_exact = waveform_all.shape[1] / sample_rate
# モデルの読み込み
model = whisper.load_model('small')

In [ ]:
# 結果を保存するファイル名
output_path = "voice_to_text.txt"
MAX_TOLERANCE = 0.3
# 書き込み用にファイルを開く
with open(output_path, "w", encoding="utf-8") as f:
  for segment, _, speaker in diarization.itertracks(yield_label=True):
    start = segment.start
    end = segment.end
    # 長さ超過時の処理
    overshoot = end - duration_exact
    if overshoot > 0:
      if overshoot <= MAX_TOLERANCE:
        end = duration_exact
      else:
        # 長さ超過が許容範囲を超える場合はスキップ
        print(f"スキップ: セグメント {end:.3f}s は音声長 {duration_exact:.3f}s を {overshoot:.3f}s 超過")
        continue
    safe_segment = Segment(start, end)
    try:
      waveform, sample_rate = audio.crop(audio_file, safe_segment)
    except Exception as e:
      print(f"cropエラー: {e}")
      continue
    # 音声認識（numpy → Whisper）
    try:
      text = model.transcribe(waveform.squeeze().numpy())["text"]
    except Exception as e:
      print(f"認識エラー: {e}")
      text = ""
    # 結果の出力
    line = f"[{safe_segment.start:03.1f}s - {safe_segment.end:03.1f}s] {speaker}: {text}"
    print(line)
    f.write(line + "\n")